# FLIGHTS INFORMATION CODE

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from selenium import webdriver

## API flights


The data related to the flights requested is obtained from an API of Aviation Edge Database ('https://aviation-edge.com/').

In [33]:
#API route
aed = 'http://aviation-edge.com/v2/public/flights?key=25034e-9edde7&flightIata='
#Flight to be requested (example)
flight = 'CA1028'
url = (f'{aed}{flight}')
print(url)
html_flight = requests.get(url).json()
html_flight[:1000]


http://aviation-edge.com/v2/public/flights?key=25034e-9edde7&flightIata=CA1028


[{'aircraft': {'iataCode': 'B77L',
   'icao24': '780C5C',
   'icaoCode': 'B77L',
   'regNumber': 'B-2092'},
  'airline': {'iataCode': 'CAO', 'icaoCode': 'CAO'},
  'arrival': {'iataCode': 'HGH', 'icaoCode': 'ZSHC'},
  'departure': {'iataCode': 'MAD', 'icaoCode': 'LEMD'},
  'flight': {'iataNumber': 'CA1028',
   'icaoNumber': 'CAO1028',
   'number': '1028'},
  'geography': {'altitude': 10363.2,
   'direction': 65.19,
   'latitude': 52.49,
   'longitude': 16.16},
  'speed': {'horizontal': 865.08, 'isGround': 0.0, 'vspeed': 0.0},
  'status': 'en-route',
  'system': {'squawk': None, 'updated': 1618942319}}]

In [23]:
type(html_flight)

list

In [25]:
html_flight_dic = html_flight[0]
html_flight_dic

{'aircraft': {'iataCode': 'B77L',
  'icao24': '780C5C',
  'icaoCode': 'B77L',
  'regNumber': 'B-2092'},
 'airline': {'iataCode': 'CAO', 'icaoCode': 'CAO'},
 'arrival': {'iataCode': 'HGH', 'icaoCode': 'ZSHC'},
 'departure': {'iataCode': 'MAD', 'icaoCode': 'LEMD'},
 'flight': {'iataNumber': 'CA1028', 'icaoNumber': 'CAO1028', 'number': '1028'},
 'geography': {'altitude': 10363.2,
  'direction': 24.33,
  'latitude': 47.34,
  'longitude': 7.74},
 'speed': {'horizontal': 871.956, 'isGround': 0.0, 'vspeed': 0.0},
 'status': 'en-route',
 'system': {'squawk': None, 'updated': 1618938720}}

In [28]:
flight_df = pd.DataFrame.from_dict(html_flight_dic)
flight_df

,aircraft,airline,arrival,departure,flight,geography,speed,status,system
iataCode,B77L,CAO,HGH,MAD,NaN,NaN,NaN,en-route,NaN
icao24,780C5C,NaN,NaN,NaN,NaN,NaN,NaN,en-route,NaN
icaoCode,B77L,CAO,ZSHC,LEMD,NaN,NaN,NaN,en-route,NaN
regNumber,B-2092,NaN,NaN,NaN,NaN,NaN,NaN,en-route,NaN
iataNumber,NaN,NaN,NaN,NaN,CA1028,NaN,NaN,en-route,NaN
icaoNumber,NaN,NaN,NaN,NaN,CAO1028,NaN,NaN,en-route,NaN
number,NaN,NaN,NaN,NaN,1028,NaN,NaN,en-route,NaN
altitude,NaN,NaN,NaN,NaN,NaN,10363.20,NaN,en-route,NaN
direction,NaN,NaN,NaN,NaN,NaN,24.33,NaN,en-route,NaN
latitude,NaN,NaN,NaN,NaN,NaN,47.34,NaN,en-route,NaN


## Airports information (dataframe)

There are three different dataframes that contains airports information. Let's see the information that is contained in each one.

In [34]:
airports = pd.read_csv('data/airports-extended.csv', header = None)
airports

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
12664,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
12665,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports
12666,14110,Melitopol Air Base,Melitopol,Ukraine,\N,UKDM,46.880001,35.305000,0,\N,\N,\N,airport,OurAirports


In [36]:
airports1 = pd.read_csv('data/airports (1).csv')
airports1

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63993,32753,ZYYY,medium_airport,Shenyang Dongta Airport,41.784401,123.496002,NaN,AS,CN,CN-21,"Dadong, Shenyang",no,ZYYY,NaN,NaN,NaN,NaN,NaN
63994,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad
63995,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN
63996,342102,ZZZW,closed,Scandium City Heliport,69.355287,-138.939310,4.0,NaN,CA,CA-YT,(Old) Scandium City,no,ZZZW,ZYW,YK96,NaN,NaN,NaN


In [45]:
world_airports = pd.read_csv('data/world-airports.csv')
world_airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,2434,EGLL,large_airport,London Heathrow Airport,51.470600,-0.461941,83.0,EU,United Kingdom,GB,...,London,1,EGLL,LHR,NaN,http://www.heathrowairport.com/,https://en.wikipedia.org/wiki/Heathrow_Airport,"LON, Londres",1251675,2019-10-19T10:16:49+00:00
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125.0,NaN,United States,US,...,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,NaN,1335475,2020-04-26T22:37:22+00:00
2,3754,KORD,large_airport,Chicago O'Hare International Airport,41.978600,-87.904800,672.0,NaN,United States,US,...,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2018-09-16T02:35:35+00:00
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639801,-73.778900,13.0,NaN,United States,US,...,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild",1052075,2021-04-10T17:02:47+00:00
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026.0,NaN,United States,US,...,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64011,324519,PL-0137,small_airport,Żłobek Mały Airfield,51.454611,23.542472,515.0,EU,Poland,PL,...,Żłobek,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2020-05-02T12:34:24+00:00
64012,42246,BITH,closed,Sauðanes Airport,66.251859,-15.273914,15.0,EU,Iceland,IS,...,Þórshöfn,0,BITH,NaN,NaN,NaN,https://en.wikipedia.org/wiki/%C3%9E%C3%B3rsh%...,"THO, Thorshofn",0,2021-03-23T00:07:10+00:00
64013,313231,HOO,small_airport,Nhon Co Airfield,11.978700,107.563800,2270.0,AS,Vietnam,VN,...,Đăk R'Lấp,0,NaN,HOO,NaN,NaN,NaN,NaN,0,2021-01-10T20:43:41+00:00
64014,323411,PL-0098,small_airport,Łagowiec,52.335250,15.665277,NaN,EU,Poland,PL,...,Łagowiec,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2017-05-24T02:23:15+00:00


In [46]:
world_airports.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'country_name', 'iso_country',
       'region_name', 'iso_region', 'local_region', 'municipality',
       'scheduled_service', 'gps_code', 'iata_code', 'local_code', 'home_link',
       'wikipedia_link', 'keywords', 'score', 'last_updated'],
      dtype='object')

We will work with world_airports dataframe, as it is the one that have more information and better ordered.

In [52]:
#Search of the city of departure
departure_code = flight_df.loc['iataCode','departure']
print(departure_code)

MAD


In [54]:
#Location of the airport with that iata code in word_airports dataframe 
departure_airport = world_airports.loc[world_airports['iata_code'] == departure_code]
departure_airport

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
31,4019,LEMD,large_airport,Adolfo Suárez Madrid–Barajas Airport,40.471926,-3.56264,1998.0,EU,Spain,ES,...,Madrid,1,LEMD,MAD,NaN,http://www.aena.es/en/passengers/passengers.html,https://en.wikipedia.org/wiki/Adolfo_Su%C3%A1r...,"Leganés, Madrid Barajas International Airport",1041675,2018-09-13T12:25:48+00:00


In [56]:
#Search of the city of arrival
arrival_code = flight_df.loc['iataCode','arrival']
print(arrival_code)

HGH


In [57]:
#Location of the airport with that iata code in word_airports dataframe 
arrival_airport = world_airports.loc[world_airports['iata_code'] == arrival_code]
arrival_airport

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
1043,27218,ZSHC,large_airport,Hangzhou Xiaoshan International Airport,30.2295,120.433998,23.0,AS,China,CN,...,Hangzhou,1,ZSHC,HGH,NaN,http://www.hzairport.com/,https://en.wikipedia.org/wiki/Hangzhou_Xiaosha...,"Hang Zhou, 杭州萧山国际机场",1050,2009-04-17T07:32:06+00:00
